In [16]:
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import tushare as ts
import pandas as pd
import numpy as np
import pymysql
import time
pd.set_option('max_colwidth',200)
%matplotlib inline

ts.set_token('2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67')

# 1. 常用函数基本信息介绍

=====fut_basic 输入参数=====

- exchange	str	Y	交易所代码 

- CFFEX-中金所 (ts_code: CFX)
- DCE-大商所 
- CZCE-郑商所 
- SHFE-上期所 
- INE-上海国际能源交易中心

fut_type	str	N	合约类型 

(1 普通合约 2主力与连续合约 默认取全部)

=====fut_basic 输出参数=====

- 名称	类型	默认显示	描述
- ts_code	str	Y	合约代码

- symbol	str	Y	交易标识
- exchange	str	Y	交易市场
- name	str	Y	中文简称
- fut_code	str	Y	合约产品代码
- multiplier	float	Y	合约乘数
- trade_unit	str	Y	交易计量单位
- per_unit	float	Y	交易单位(每手)
- quote_unit	str	Y	报价单位
- quote_unit_desc	str	Y	最小报价单位说明
- d_mode_desc	str	Y	交割方式说明
- list_date	str	Y	上市日期
- delist_date	str	Y	最后交易日期
- d_month	str	Y	交割月份
- last_ddate	str	Y	最后交割日
- trade_time_desc	str	N	交易时间说明

https://tushare.pro/document/2?doc_id=135

=====fut_settle 输入参数=====

名称	类型	必选	描述

- trade_date	str	N	交易日期 （trade_date/ts_code至少需要输入一个参数）
- ts_code	str	N	合约代码
- start_date	str	N	开始日期
- end_date	str	N	结束日期
- exchange	str	N	交易所代码

=====fut_settle 输出参数=====

名称	类型	默认显示	描述

- ts_code	str	Y	合约代码
- trade_date	str	Y	交易日期
- settle	float	Y	结算价
- trading_fee_rate	float	Y	交易手续费率
- trading_fee	float	Y	交易手续费
- delivery_fee	float	Y	交割手续费
- b_hedging_margin_rate	float	Y	买套保交易保证金率
- s_hedging_margin_rate	float	Y	卖套保交易保证金率
- long_margin_rate	float	Y	买投机交易保证金率
- short_margin_rate	float	Y	卖投机交易保证金率
- offset_today_fee	float	N	平今仓手续率
- exchange	str	N	交易所

https://tushare.pro/document/2?doc_id=141

# 2. Tushare期货数据：

https://tushare.pro/document/2?doc_id=134



- 数据名称	API	描述	需要最低积分	每分钟次数

- 期货合约列表	fut_basic	全部历史	200	80

- 期货交易日历	trade_cal	数据开始月1996年1月，定期更新	0	无限制

- 期货日线行情	fut_daily	数据开始月1996年1月，每日盘后更新	200	120

- 每日成交持仓排名	fut_holding	数据开始月2002年1月，每日盘后更新	600	80

- 仓单日报	fut_wsr	数据开始月2006年1月，每日盘后更新	600	80

- 结算参数	fut_settle	数据开始月2012年1月，每日盘后更新	600	80

=================================

- 主力合约	连续合约	普通合约

- XX	XXL	XXMMDD

- 例如：CU.SHF	例如：CUL.SHF	例如：CU1811.SHF

In [2]:
db_ip = '127.0.0.1'
db_user = 'root'#用户
# db_password = '1q2w3e4rasdf'#密码
db_password = None
db_name = 'market_data'#数据库
db_port = "3306"

conn = pymysql.connect(db_ip,
                       db_user,
                       db_password,
                       db_name,
                       autocommit = 1)
cursor = conn.cursor()
engine = create_engine("mysql+pymysql://root:@localhost:3306/market_data")

OperationalError: (2003, "Can't connect to MySQL server on '127.0.0.1' ([WinError 10061] 由于目标计算机积极拒绝，无法连接。)")

# 5. 调用分钟数据 - 通用行情接口


## 5.2 循环调取数据

### https://tushare.pro/document/2?doc_id=109

In [23]:
df = ts.pro_bar(ts_code='000001.SH', asset='I', start_date='20180101', end_date='20181011')

In [24]:
df

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,000001.SH,20181011,2583.4575,2643.0740,2661.2859,2560.3164,2725.8367,-142.3792,-5.2233,197150702.0,170057762.5
1,000001.SH,20181010,2725.8367,2723.7242,2743.5480,2703.0626,2721.0130,4.8237,0.1773,113485736.0,111312455.3
2,000001.SH,20181009,2721.0130,2713.7319,2734.3142,2711.1971,2716.5104,4.5026,0.1657,116771899.0,110292457.8
3,000001.SH,20181008,2716.5104,2768.2075,2771.9384,2710.1781,2821.3501,-104.8397,-3.7159,149501388.0,141531551.8
4,000001.SH,20180928,2821.3501,2794.2644,2821.7553,2791.8363,2791.7748,29.5753,1.0594,134290456.0,125369989.4
5,000001.SH,20180927,2791.7748,2805.7930,2810.5062,2786.3217,2806.8133,-15.0385,-0.5358,123997696.0,123447711.6
6,000001.SH,20180926,2806.8133,2785.3165,2827.3413,2783.3419,2781.1385,25.6748,0.9232,139567706.0,145058769.4
7,000001.SH,20180925,2781.1385,2775.0663,2790.0321,2771.1595,2797.4848,-16.3463,-0.5843,111983179.0,108528047.8
8,000001.SH,20180921,2797.4848,2733.8742,2797.4851,2722.0319,2729.2438,68.2410,2.5004,158016642.0,148976357.2
9,000001.SH,20180920,2729.2438,2732.1697,2743.9645,2724.0835,2730.8503,-1.6065,-0.0588,111387463.0,104875868.9


In [29]:
# test = ts.pro_bar(ts_code='IF1909.CFX',freq="1min" ,start_date="2017-02-20 09:00:00", end_date="2019-10-06 9:00:00")
test = ts.pro_bar(ts_code='IF1910.CFX',freq="D" , asset="FT", start_date="20170220", end_date="20191006")

In [30]:
test

,ts_code,trade_date,pre_close,pre_settle,open,high,low,close,settle,change1,change2,vol,amount,oi,oi_chg
0,IF1910.CFX,20190930,3855.8,3856.6,3850.0,3859.4,3792.6,3794.0,3822.2,-62.6,-34.4,60415.0,6958045.740,69224.0,3178.0
1,IF1910.CFX,20190927,3844.4,3852.6,3852.2,3864.2,3840.2,3855.8,3856.6,3.2,4.0,51720.0,5977333.080,66046.0,-6767.0
2,IF1910.CFX,20190926,3869.6,3880.6,3885.0,3895.4,3841.8,3844.4,3852.6,-36.2,-28.0,70645.0,8201792.748,72813.0,3061.0
3,IF1910.CFX,20190925,3894.6,3899.4,3884.6,3894.0,3868.6,3869.6,3880.6,-29.8,-18.8,66731.0,7769331.762,69752.0,-3639.0
4,IF1910.CFX,20190924,3894.8,3884.2,3902.0,3934.0,3889.2,3894.6,3899.4,10.4,15.2,76051.0,8916199.560,73391.0,1846.0
5,IF1910.CFX,20190923,3926.0,3925.2,3912.2,3913.0,3867.6,3894.8,3884.2,-30.4,-41.0,70709.0,8233350.474,71545.0,4987.0
6,IF1910.CFX,20190920,3921.0,3913.6,3929.8,3939.0,3918.8,3926.0,3925.2,12.4,11.6,59557.0,7015585.680,66558.0,14739.0
7,IF1910.CFX,20190919,3901.2,3906.4,3919.0,3922.4,3890.2,3921.0,3913.6,14.6,7.2,40587.0,4758730.392,51819.0,21827.0
8,IF1910.CFX,20190918,3887.2,3890.2,3900.4,3922.0,3898.8,3901.2,3906.4,11.0,16.2,18846.0,2209960.590,29992.0,5337.0
9,IF1910.CFX,20190917,3958.4,3951.6,3955.8,3955.8,3875.2,3887.2,3890.2,-64.4,-61.4,17675.0,2075566.506,24655.0,7865.0


In [31]:
test2 = ts.pro_bar(ts_code='IF1909.CFX',freq="D" , asset="FT", start_date="20170220", end_date="20191006")

In [32]:
test2

,ts_code,trade_date,pre_close,pre_settle,open,high,low,close,settle,change1,change2,vol,amount,oi,oi_chg
0,IF1909.CFX,20190920,3925.0,3916.0,3930.6,3939.4,3922.0,3932.0,3932.45,16.0,16.45,19946.0,2.351285e+06,0.0,-22343.0
1,IF1909.CFX,20190919,3904.2,3908.0,3916.6,3925.0,3893.2,3925.0,3916.00,17.0,8.00,44195.0,5.184377e+06,22343.0,-23663.0
2,IF1909.CFX,20190918,3888.4,3890.2,3903.4,3922.0,3898.6,3904.2,3908.00,14.0,17.80,55124.0,6.466370e+06,46006.0,-17839.0
3,IF1909.CFX,20190917,3959.8,3953.4,3949.8,3951.8,3875.6,3888.4,3890.20,-65.0,-63.20,77870.0,9.147640e+06,63845.0,-5225.0
4,IF1909.CFX,20190916,3976.8,3975.6,3976.8,3978.8,3938.6,3959.8,3953.40,-15.8,-22.20,66673.0,7.922083e+06,69070.0,-6036.0
5,IF1909.CFX,20190912,3933.8,3936.2,3953.4,3987.6,3942.0,3976.8,3975.60,40.6,39.40,72082.0,8.564692e+06,75106.0,-7940.0
6,IF1909.CFX,20190911,3960.6,3958.0,3969.0,3972.6,3926.0,3933.8,3936.20,-24.2,-21.80,80275.0,9.497691e+06,83046.0,2530.0
7,IF1909.CFX,20190910,3968.2,3963.8,3976.2,3978.0,3938.2,3960.6,3958.00,-3.2,-5.80,67884.0,8.048599e+06,80516.0,-5830.0
8,IF1909.CFX,20190909,3958.0,3942.8,3980.0,3983.8,3952.6,3968.2,3963.80,25.4,21.00,77803.0,9.254348e+06,86346.0,-3450.0
9,IF1909.CFX,20190906,3923.6,3929.4,3941.0,3958.0,3928.2,3958.0,3942.80,28.6,13.40,77323.0,9.142272e+06,89796.0,-3801.0


In [18]:
import csv

In [31]:
def write_csv(file_name, headers, rows):
    with open(file_name, 'w')as f:
        f_csv = csv.DictWriter(f, headers)
        f_csv.writeheader()  # 如果需要写入标题就可以
        f_csv.writerows(rows)
        f.close()
        
file_name ='ok.csv'
headers = ['企业信息', '人员信息']
rows = [{'企业信息':'c_detail', '人员信息':'123'}]

write_csv(file_name, headers, rows)

In [21]:
write_csv("test.csv", None, test.iloc[0, :])

AttributeError: 'str' object has no attribute 'keys'

In [12]:
date_list = list(pd.date_range("2019-02-18 09:00:00", "2019-02-23 9:00:00").astype(str))
for i in range(len(date_list)):
    df_list = []
    try:
        df_date = ts.pro_bar(ts_code='IF1909.CFX',freq="1min" ,start_date=date_list[i], end_date=date_list[i+1])
        df_date.sort_values("trade_time", inplace = True)
        df_date.reset_index(inplace =True)
    except:
#         elapsed = (time.process_time() - start)
#         print("Time used for saving.:",elapsed)
        print("This Batch is Finished.")
        break
    df_list.append(df_date)
    df_sample = pd.concat(df_list)
    del df_sample['index']
    df_sample.reset_index(inplace = True)
    del df_sample['index']
    df_sample = df_sample[['trade_time','ts_code','open','high','low','close','vol','amount','trade_date','pre_close']]
    print("Data Processed for {}.".format(date_list[i]))
    
     #     start = time.process_time()
    for i in range(len(df_sample)):
        val0 = df_sample.iloc[i, :10][0]
        val1 = df_sample.iloc[i, :10][1]
        val2 = int(df_sample.iloc[i, :10][2])
        val3 = int(df_sample.iloc[i, :10][3])
        val4 = int(df_sample.iloc[i, :10][4])
        val5 = int(df_sample.iloc[i, :10][5])
        val6 = int(df_sample.iloc[i, :10][6])
        val7 = int(df_sample.iloc[i, :10][7])
        val8 = df_sample.iloc[i, :10][8]
        try:
            cursor.execute("""INSERT INTO futures_data (trade_time,ts_code,open,high,low,close,vol,amount,trade_date) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)""",(val0,val1,val2,val3,val4,val5,val6,val7,val8))
            print("第{}行 INSERTED".format(i))
        except Exception as e:
            print(e)
    print("Chill and relax for 10 seconds...")
    time.sleep(11)
    print("Let's get started again.")

Data Processed for 2019-02-18 09:00:00.
Data Processed for 2019-02-19 09:00:00.
Data Processed for 2019-02-20 09:00:00.
Data Processed for 2019-02-21 09:00:00.
抱歉，您每分钟最多访问该接口5次，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。
This Batch is Finished.


In [23]:
df_sample.head(1)

,trade_time,ts_code,open,high,low,close,vol,amount,trade_date,pre_close
0,2019-02-21 09:30:00,IF1909.CFX,3433.0,3433.0,3433.0,3433.0,0.0,0.0,20190221,NaN


In [34]:
def write_csv(file_name, headers, rows):
    with open(file_name, 'w')as f:
        f_csv = csv.DictWriter(f, headers)
        f_csv.writeheader()  # 如果需要写入标题就可以
        f_csv.writerows(rows)
        f.close()

In [33]:
row_dict = {}
for i in range(len(df_sample.head(5))):
    val0 = df_sample.iloc[i, :10][0]
    val1 = df_sample.iloc[i, :10][1]
    val2 = int(df_sample.iloc[i, :10][2])
    val3 = int(df_sample.iloc[i, :10][3])
    val4 = int(df_sample.iloc[i, :10][4])
    val5 = int(df_sample.iloc[i, :10][5])
    val6 = int(df_sample.iloc[i, :10][6])
    val7 = int(df_sample.iloc[i, :10][7])
    val8 = df_sample.iloc[i, :10][8]
    
    row_dict["trade_time"] = val0
    row_dict["ts_code"] = val1
    row_dict["open"] = val2
    row_dict["high"] = val3
    row_dict["low"] = val4
    row_dict["close"] = val5
    row_dict["vol"] = val6
    row_dict["amount"] = val7
    row_dict["trade_date"] = val8
    file_name = "test_write_20191005.csv"
    headers = ['trade_time','ts_code','open','high','low','close','vol','amount','trade_date']
    rows = row_dict
    print(rows)
    write_csv(file_name, headers, rows)

{'trade_time': '2019-02-21 09:30:00', 'ts_code': 'IF1909.CFX', 'open': 3433, 'high': 3433, 'low': 3433, 'close': 3433, 'vol': 0, 'amount': 0, 'trade_date': '20190221'}


AttributeError: 'str' object has no attribute 'keys'

# PYTHON 读、写、追加CSV文件详细以及注意事项

https://www.cnblogs.com/hoojjack/p/9670715.html

In [2]:
import os

def create_csv(path, head):
    with open(path, "w+", newline='') as file:
        csv_file = csv.writer(file)
#         head = ["name","sex"]
        csv_file.writerow(head)

def append_csv(path, data):
    with open(path, "a+", newline='') as file: # 处理csv读写时不同换行符  linux:\n    windows:\r\n    mac:\r
        csv_file = csv.writer(file)
#         data = [val0, val1, val2, val3, val4, val5, val6, val7, val8]
        csv_file.writerows(data)
    
def execute(path):
    if os.path.exists(path):
        append_csv(path, data)
    else:
        create_csv(path, head)

# def read_csv(path):
#     with open(path,"r+") as file:
#         csv_file = csv.reader(file)
#         for data in csv_file:
#             print("data:", data)
        
path = "example_20191006.csv"
head = ['trade_time','ts_code','open','high','low','close','vol','amount','trade_date']
date_list = list(pd.date_range("2019-02-18 09:00:00", "2019-02-20 9:00:00").astype(str))

In [70]:
for i in range(len(date_list)):
    df_list = []
    try:
        df_date = ts.pro_bar(ts_code='IF1909.CFX',freq="1min" ,start_date=date_list[i], end_date=date_list[i+1])
        df_date.sort_values("trade_time", inplace = True)
        df_date.reset_index(inplace =True)
    except:
#         elapsed = (time.process_time() - start)
#         print("Time used for saving.:",elapsed)
        print("This Batch is Finished.")
        break
    df_list.append(df_date)
    df_sample = pd.concat(df_list)
    del df_sample['index']
    df_sample.reset_index(inplace = True)
    del df_sample['index']
    df_sample = df_sample[['trade_time','ts_code','open','high','low','close','vol','amount','trade_date','pre_close']]
    print("Data Processed for {}.".format(date_list[i]))

    for i in range(len(df_sample.head(5))):
        val0 = df_sample.iloc[i, :10][0]
        val1 = df_sample.iloc[i, :10][1]
        val2 = int(df_sample.iloc[i, :10][2])
        val3 = int(df_sample.iloc[i, :10][3])
        val4 = int(df_sample.iloc[i, :10][4])
        val5 = int(df_sample.iloc[i, :10][5])
        val6 = int(df_sample.iloc[i, :10][6])
        val7 = int(df_sample.iloc[i, :10][7])
        val8 = df_sample.iloc[i, :10][8]

        data = [[val0, val1, val2, val3, val4, val5, val6, val7, val8]]

        if __name__ == "__main__":
            execute(path)
            
    print("Chill and relax for 13 seconds...")
    time.sleep(13)
    print("Let's get started again.")

Data Processed for 2019-02-18 09:00:00.
Chill and relax for 12 seconds...
Let's get started again.
Data Processed for 2019-02-19 09:00:00.
Chill and relax for 12 seconds...


KeyboardInterrupt: 